### Matrix Factorization and Dimension Reducation Techniques

This notebook is the exercise for matrix factorization methods (SVD) used in recommender system. 

Again we start from the rating matrix. Large rating matrix is an overfit representation of user tastes and item descriptions:

* leads to problems of synonymy
* leads to computational complexity, potentially poor results

An ideal alternative would be to have a more compact representation of user tastes and item descriptions.

SVD (Singular Value Decomposition) is a popular matrix decomposition method. In information retrieveal community had same issue and addressed this: key word vectors had problem that queries and documents were poorly represented, they want "concepts" rather than "words". And SVD can reduce space to a smaller taste space that is compact and robust.

<img src="./svd-matrices.png">

Here we have the rating matrix \\(R\\) with \\(m\\) users and \\(n\\) items, we can factorize it

$$R = P\Sigma Q^\top,$$

where the rating matrix is decomposited as multiplication of \\(P\\) - user - feature affinity matrix, \\(\Sigma\\) - feature weight matrix and \\(Q\\) - item - feature relevance matrix. And \\(P\\) and \\(Q\\) are orthogonal if the SVD is done correctly. And this decomposition is exists for any real matrix.

By choosing top \\(k\\) values in \\(\Sigma\\), we are using Truncated SVD, and could get the approximation of the raw matrix

$$R_{k} = P_{k}\Sigma_{k} Q^\top_{k}.$$

In this way the dimension is reduced and we get small results, but the exact value of \\(k\\) need to be exprimented.

Then in application, for example predict the rating of a item \\(i\\) by a user \\(u\\), pick the **row** vector from \\(P\\), multiply by matrix \\(\Sigma\\), then multiply the **column** vector from \\(Q\\), the final scalar value is the prediction. And note that this is not same as Content Based Filtering since the rows and columns are not "interpretable", it defines a shared vector space. The challenge of SVD is also obvious:

* how to deal with missing values
* time complexity
* lack of explainbility, we call the features to be latent features

In real applications, as new data comes, we can use "data folding-in". That means using existing factorization to compute the predictions for new data. But this only works if the SVD is algebraically correct, which means \\(P\\) and \\(Q\\) are orthogonal. The situation that SVD not "correct" is when we use **gradient descent**. I will update this part in future posts.

In [1]:
import numpy as np
import pandas as pd

In [2]:
item_vectors = pd.read_excel('Assignment 6.xlsx', sheet_name='Items')
user_vectors = pd.read_excel('Assignment 6.xlsx', sheet_name='Users', index_col='User')

singular_val = item_vectors.iloc[0]
item_vectors = item_vectors.iloc[2:]
print('Number of features = {}'.format(len(singular_val)))
print('Number of users = {}'.format(user_vectors.shape[0]))
print('Number of items = {}'.format(item_vectors.shape[0]))

Number of features = 15
Number of users = 25
Number of items = 100


Get top 5 movies for 1st and 2nd features.

In [3]:
item_vectors.iloc[:, :2].sort_values(by=1, ascending=False).head(5)

,,1,2
4327,Charlie's Angels (2000),0.281990,0.080262
414,Batman Forever (1995),0.218089,-0.009324
3049,Ace Ventura: Pet Detective (1994),0.190879,0.067747
8467,Dumb & Dumber (1994),0.190638,0.110683
854,The Mask (1994),0.158601,-0.006160


In [4]:
item_vectors.iloc[:, :2].sort_values(by=2, ascending=False).head(5)

,,1,2
14,American Beauty (1999),-0.044468,0.198715
680,Pulp Fiction (1994),-0.166983,0.189565
24,Kill Bill: Vol. 1 (2003),-0.045203,0.181570
275,Fargo (1996),-0.043682,0.161559
38,Eternal Sunshine of the Spotless Mind (2004),-0.048743,0.161058


Make a prediction of user 4469, this is simply matrix multiplication:

$$\hat{r} = \sum_{f} p_{uf}s_{f}q_{fi}.$$

In [5]:
user_id = 4469
rating_preds = user_vectors.loc[user_id].dot(np.diag(singular_val)).dot(item_vectors.T)
rating_preds_df = pd.DataFrame(rating_preds, columns=['rating_preds'])
rating_preds_df.index = [item_vectors.index[i][0] for i in range(100)]
rating_preds_df.sort_values(by='rating_preds', ascending=False).head(5)

,rating_preds
278,0.207680
453,0.183286
98,0.173611
238,0.172180
13,0.170744
